In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This notebook is inspired by the following werus23's amazing notebooks.
import pandas as pd
import numpy as np
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
import joblib
import random
import math
from tqdm.auto import tqdm 

from scipy.interpolate import interp1d

from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa 
import ctypes
torch.set_num_interop_threads(4)
torch.set_num_threads(4)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#data loading,cleaning and optimization
class PATHS:
    MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
    # CSV FILES : 
    SUBMISSION = MAIN_DIR + "sample_submission.csv"
    TRAIN_EVENTS = MAIN_DIR + "train_events.csv"
    # PARQUET FILES:
    TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
    TEST_SERIES = MAIN_DIR + "test_series.parquet"
class CFG:
    DEMO_MODE = True
class data_reader:
    def __init__(self, demo_mode):
        super().__init__()
        # MAPPING FOR DATA LOADING :
        self.names_mapping = {
            "submission" : {"path" : PATHS.SUBMISSION, "is_parquet" : False, "has_timestamp" : False}, 
            "train_events" : {"path" : PATHS.TRAIN_EVENTS, "is_parquet" : False, "has_timestamp" : True},
            "train_series" : {"path" : PATHS.TRAIN_SERIES, "is_parquet" : True, "has_timestamp" : True},
            "test_series" : {"path" : PATHS.TEST_SERIES, "is_parquet" : True, "has_timestamp" : True}
        }
        self.valid_names = ["submission", "train_events", "train_series", "test_series"]
        self.demo_mode = demo_mode
    
    def verify(self, data_name):
        "function for data name verification"
        if data_name not in self.valid_names:
            print("PLEASE ENTER A VALID DATASET NAME, VALID NAMES ARE : ", valid_names)
        return
    
    def cleaning(self, data):
        "cleaning function : drop na values"
        before_cleaning = len(data)
        print("Number of missing timestamps : ", len(data[data["timestamp"].isna()]))
        data = data.dropna(subset=["timestamp"])
        after_cleaning = len(data)
        print("Percentage of removed rows : {:.1f}%".format(100 * (before_cleaning - after_cleaning) / before_cleaning) )
#         print(data.isna().any())
#         data = data.bfill()
        return data
    
    @staticmethod
    def reduce_memory_usage(data):
        "iterate through all the columns of a dataframe and modify the data type to reduce memory usage."
        start_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
        for col in data.columns:
            col_type = data[col].dtype    
            if col_type != object:
                c_min = data[col].min()
                c_max = data[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        data[col] = data[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        data[col] = data[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        data[col] = data[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        data[col] = data[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        data[col] = data[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        data[col] = data[col].astype(np.float32)
                    else:
                        data[col] = data[col].astype(np.float64)
            else:
                data[col] = data[col].astype('category')

        end_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        return data
    
    def load_data(self, data_name):
        "function for data loading"
        self.verify(data_name)
        data_props = self.names_mapping[data_name]
        if data_props["is_parquet"]:
            if self.demo_mode:
                pf = ParquetFile(data_props["path"]) 
                demo_rows = next(pf.iter_batches(batch_size=20_000)) 
                data = pa.Table.from_batches([demo_rows]).to_pandas()
            else:
                data = pd.read_parquet(data_props["path"])
        else:
            if self.demo_mode:
                data = pd.read_csv(data_props["path"], nrows=20_000)
            else:
                data = pd.read_csv(data_props["path"])
                
        gc.collect()
        if data_props["has_timestamp"]:
            print('cleaning')
            data = self.cleaning(data)
            gc.collect()
        data = self.reduce_memory_usage(data)
        return data

In [ ]:
reader = data_reader(demo_mode=False)
test_series = reader.load_data(data_name="test_series")
ids = test_series.series_id.unique()
gc.collect()

In [ ]:
#wavenet model
class Wave_Block(nn.Module):

    def __init__(self, in_channels, out_channels, dilation_rates, kernel_size):
        super(Wave_Block, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()

        self.convs.append(nn.Conv1d(in_channels, out_channels, kernel_size=1))     # 1 D convolution with the input size of (N,Cin,L) N is batch size, C denote the number of channels,L sequence length
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.gate_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1))

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x)) * torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i + 1](x)
            res = res + x
        return res
# detail 
class Classifier(nn.Module):
    def __init__(self, inch=10):
        super().__init__()
        #self.LSTM = nn.GRU(input_size=16, hidden_size=16, num_layers=1, batch_first=True, bidirectional=False)
        self.wave_block1 = Wave_Block(inch, 16, 12, 3)
        self.wave_block2 = Wave_Block(16, 32, 8, 3)
        self.wave_block3 = Wave_Block(32, 64, 4, 3)
        self.wave_block4 = Wave_Block(64, 128, 1, 3)
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)    # 1 D convolution with the input size of (N,Cin,L) N is batch size, C denote the number of channels,L sequence length
        x = self.wave_block1(x)
        x = self.wave_block2(x)
        x = self.wave_block3(x)

        x = self.wave_block4(x)      #1D convolution output is (N,Cout,L) but linear layer accepts the input to be in (batch_size, num_channels) so using the permute before the linear layer gives(batch_size, sequence_length, num_channels). The actual number of features for the linear layer is determined by the last dimension after the permute, which is num_channels. So, it's more accurate to say that the linear layer expects an input in the form of (batch_size, num_channels) based on how the data is arranged before the linear layer.
        x = x.permute(0, 2, 1)
        #print(x.shape)
        #x, _ = self.LSTM(x)          #Assuming you have an input sequence of shape (batch_size, sequence_length, input_size) and you pass this sequence through an LSTM layer, you will get an output of shape (batch_size, sequence_length, num_directions=2(bidirectional) * hidden_size),
        x = self.fc(x)
        
        return x

In [ ]:
class SleepDataset(Dataset):
    def __init__(
        self,
        series_ids,
        series,
    ):
        series_ids = series_ids
        series = series.reset_index()
        self.data = []
        
        for viz_id in tqdm(series_ids):
            self.data.append(series.loc[(series.series_id==viz_id)].copy().reset_index())
            
    def downsample_seq_generate_features(self,feat, downsample_factor):
        
        if len(feat)%12!=0:
            feat = np.concatenate([feat,np.zeros(12-((len(feat))%12))+feat[-1]])
        feat = np.reshape(feat, (-1,12))
        feat_mean = np.mean(feat,1)
        feat_std = np.std(feat,1)
        feat_median = np.median(feat,1)
        feat_max = np.max(feat,1)
        feat_min = np.min(feat,1)

        return np.dstack([feat_mean,feat_std,feat_median,feat_max,feat_min])[0]
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index][['anglez','enmo']].values.astype(np.float32)
        X = np.concatenate([self.downsample_seq_generate_features(X[:,i],12) for i in range(X.shape[1])],-1)
        X = torch.from_numpy(X)
        return X
test_ds = SleepDataset(test_series.series_id.unique(),test_series)
del test_series
gc.collect()

In [ ]:
max_chunk_size = 24*60*100
min_interval = 30

In [ ]:
# model tested on test data 
model =  Classifier().to(device).eval()
model.load_state_dict(torch.load(f'/kaggle/input/compt-train2/model_best.pth',map_location=device))
submission = pd.DataFrame()
for i in range(len(test_ds)):
    X = test_ds[i].half()
    XS=X.reshape(1,X.shape[0],X.shape[1])
    seq_len = X.shape[0]
    
    pred = torch.zeros((len(X),2)).half()
    for j in range(0, seq_len, max_chunk_size):
        y_pred= model(XS[j: j + max_chunk_size].float())
        
        pred[j : j + max_chunk_size] = y_pred.detach()
        del y_pred;gc.collect()
    del X;gc.collect()
    pred = pred.numpy()
    
    series_id = ids[i]
    
    days = len(pred)/(17280/12)
    scores0,scores1 = np.zeros(len(pred),dtype=np.float16),np.zeros(len(pred),dtype=np.float16)
    for index in range(len(pred)):
        if pred[index,0]==max(pred[max(0,index-min_interval):index+min_interval,0]):
            scores0[index] = max(pred[max(0,index-min_interval):index+min_interval,0])
        if pred[index,1]==max(pred[max(0,index-min_interval):index+min_interval,1]):
            scores1[index] = max(pred[max(0,index-min_interval):index+min_interval,1])
    candidates_onset = np.argsort(scores0)[-max(1,round(days)):]
    candidates_wakeup = np.argsort(scores1)[-max(1,round(days)):]
    
    onset = test_ds.data[i][['step']].iloc[np.clip(candidates_onset*12,0,len(test_ds.data[i])-1)].astype(np.int32)
    onset['event'] = 'onset'
    onset['series_id'] = series_id
    onset['score']= scores0[candidates_onset]
    wakeup = test_ds.data[i][['step']].iloc[np.clip(candidates_wakeup*12,0,len(test_ds.data[i])-1)].astype(np.int32)
    wakeup['event'] = 'wakeup'
    wakeup['series_id'] = series_id
    wakeup['score']= scores1[candidates_wakeup]
    submission = pd.concat([submission,onset,wakeup],axis=0)
    del onset,wakeup,candidates_onset,candidates_wakeup,scores0,scores1,pred,series_id,
    gc.collect()
submission = submission.sort_values(['series_id','step']).reset_index(drop=True)
submission['row_id'] = submission.index.astype(int)
submission['score'] = submission['score'].fillna(submission['score'].mean())
submission = submission[['row_id','series_id','step','event','score']]
submission.to_csv('submission.csv',index=False)

In [ ]:
submission